# Wordle Bot
Automatyzacja gry w Wordle po angielsku (`https://wordly.org`). 

In [2]:
import selenium as se
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from pandas import DataFrame, read_csv
import numpy as np

In [3]:
driver=webdriver.Firefox()
driver.get("https://wordly.org")

Pozbycie się okienka odnośnie zbierania danych

In [4]:
manage_option=driver.find_element(by=By.CLASS_NAME, value="fc-cta-manage-options")
manage_option.click()
confirm_choices=driver.find_element(by=By.CLASS_NAME, value="fc-confirm-choices")
confirm_choices.click()
#TODO:
# - dodać odrzucenie wyborów


In [36]:
def is_word_correct(word:str,info_list:list[tuple[str,int]])->bool:
    for index,i in enumerate(info_list):
        if i[1]==0:
            for char in word:
                if char==i[0].lower():
                    return False
        elif i[1]==1:
            res=False
            for j,char in enumerate(word):
                if index==j and char==i[0].lower():
                    return False
                if index!=j and char==i[0].lower():
                    res=True
            if not res:
                return False
        else:
            if word[index]!=i[0].lower():
                return False
    return True
                
#TODO:
# uwzględnij duplikaty w literach

# Strategia początkowa - CRANE

In [107]:
turn=0
word_list=["crane"]

Wpisanie słowa

In [124]:
input_word=word_list[0]
keyboard=driver.find_element(by=By.CLASS_NAME, value="Game-keyboard")
letter_buttons=keyboard.find_elements(by=By.CLASS_NAME, value="Game-keyboard-button")
enter_button=None
for button in letter_buttons:
    if button.text=="Enter":
        enter_button=button
if enter_button is None:
    raise ValueError
for letter in input_word:
    print(letter)
    for button in letter_buttons:
        if button.text.lower()==letter:
            print(button.rect)
            button.click()
enter_button.click()


n
{'x': 729.5166625976562, 'y': 564.0, 'width': 51.44999694824219, 'height': 46.0}
y
{'x': 643.3333129882812, 'y': 460.0, 'width': 54.0, 'height': 46.0}
m
{'x': 786.9666748046875, 'y': 564.0, 'width': 51.45001220703125, 'height': 46.0}
p
{'x': 883.3333129882812, 'y': 460.0, 'width': 54.0, 'height': 46.0}
h
{'x': 676.6666870117188, 'y': 512.0, 'width': 60.666656494140625, 'height': 46.0}


Przechwycenie wyniku

In [121]:
placement_value={
    "letter-absent":0,
    "letter-elsewhere":1,
    "letter-correct":2
}
rows=driver.find_element(by=By.CLASS_NAME,value="game_rows").find_elements(by=By.CLASS_NAME,value="Row-locked-in")
print(len(rows))
info=[]
print(turn)
row_letters=rows[turn].find_elements(by=By.CLASS_NAME,value="Row-letter")
print(row_letters[0].get_attribute("class"))
for row_letter in row_letters:
    print(row_letter.rect)
    print(row_letter.get_attribute("class"))
    letter=row_letter.text.lower()
    placement_class=row_letter.get_attribute("class").split(" ")[1]
    info.append((letter,placement_value[placement_class]))
    print(info)
print(info)

    
    


3
2
Row-letter letter-correct
{'x': 488.3333129882813, 'y': 182.0, 'width': 56.0, 'height': 56.0}
Row-letter letter-correct
[('n', 2)]
{'x': 550.3333129882812, 'y': 182.0, 'width': 56.0, 'height': 56.0}
Row-letter letter-absent
[('n', 2), ('u', 0)]
{'x': 612.3333129882812, 'y': 182.0, 'width': 56.0, 'height': 56.0}
Row-letter letter-absent
[('n', 2), ('u', 0), ('t', 0)]
{'x': 674.3333129882812, 'y': 182.0, 'width': 56.0, 'height': 56.0}
Row-letter letter-absent
[('n', 2), ('u', 0), ('t', 0), ('t', 0)]
{'x': 736.3333129882812, 'y': 182.0, 'width': 56.0, 'height': 56.0}
Row-letter letter-elsewhere
[('n', 2), ('u', 0), ('t', 0), ('t', 0), ('y', 1)]
[('n', 2), ('u', 0), ('t', 0), ('t', 0), ('y', 1)]


Załadowanie listy słów

In [ ]:
if turn==0:
    word_list=read_csv("./word-bank.csv") 
    word_list=word_list.values.flatten()
    print(word_list)
    print(len(word_list))
    # valid words ma więcej liter, sprawdzić czy word bank styknie 

Filtracja listy słów

In [123]:
new_list=[]
for word in word_list:
    if is_word_correct(word,info):
        new_list.append(word)
word_list=np.array(new_list)
print(new_list)
print(len(new_list))


[np.str_('nymph')]
1


Następna tura

In [120]:
turn+=1

powtarzamy aż zwyciężymy